In [6]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('Twitter/twhin-bert-base')
model = AutoModel.from_pretrained('Twitter/twhin-bert-base')

Some weights of the model checkpoint at Twitter/twhin-bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weig

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [9]:
%%timeit
inputs = tokenizer("I'm using TwHIN-BERT! #TwHIN-BERT #NLP", return_tensors="pt")
outputs = model(**inputs)

49.5 ms ± 5.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [15]:
import os
import json
import sys
from tqdm import tqdm

In [19]:
base = "../data/train" 
files = [os.path.join(base, f) for f in os.listdir(base)]

In [26]:
def json_file_gen(paths):
    for path in tqdm(paths):
        with open(path) as f:
            yield json.load(f)

def playlist_gen(itr):
    for data in itr:
        for pl in data["playlists"]:
            yield pl

In [47]:
import torch
import numpy as np

BS = 200
PL_PER_FILE = 50000
idx = 0
buffer = []
results = {}

@torch.no_grad()
def flush():
    global buffer
    global results
    names, pids = list(zip(*buffer))
    res = model(**tokenizer(names, padding=True,return_tensors="pt")).pooler_output.float().numpy()
    for pid, z in zip(pids, res):
        results[pid] = z
    buffer = []

for i, pl in enumerate(playlist_gen(json_file_gen(files))):
    name = pl["name"]
    pid = pl["pid"]
    buffer.append((name, pid))
    if len(buffer) >= BS:
        flush()
    if len(results) >= PL_PER_FILE:
        np.save(f"../models/title_embeddings/part_{PL_PER_FILE * idx}_{PL_PER_FILE*(1+idx)}.npy", results)
        results = {}
        idx += 1

100%|█████████████████████████████████████| 1000/1000 [2:21:52<00:00,  8.51s/it]


In [60]:
np.save(f"../models/title_embeddings/part_{PL_PER_FILE * idx}_{PL_PER_FILE*(1+idx)}.npy", results)
results = {}
idx += 1

In [68]:
import numpy as np

d = np.load("../models/title_embeddings/part_150000_200000.npy", allow_pickle=True)

/home/mbh/.venvs/base/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3382: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if await self.run_code(code, result, async_=asy):


In [81]:
pls = 

(50000, 768)